In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/janestreet/data.parquet


This notebook's approach is using LSTM for times-series method.

* Ver 1: XGB
* Ver 2: PCA + XGB


In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings ("ignore")
import gc  
import xgboost as xgb
import optuna
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
SEED = 1111
tf.random.set_seed(SEED)
np.random.seed(SEED)

In [3]:
# this data is already excluded day <= 85

data = pd.read_parquet('../input/janestreet/data.parquet')

# Select trade with weight !=  0:
data = data[data['weight'] != 0]
# # limit memory use: we change datatype from float64 to float32
data = data.astype({c: np.float32 for c in data.select_dtypes(include='float64').columns}) 

# create target variable
data['action'] = (data['resp'] > 0)*1

# fill null values with mean of each feature
data.fillna(data.mean(),inplace=True)

#create fetures list
features = [c for c in data.columns if 'feature' in c]

In [4]:
# 20 / 80 split
df_trainvalid, df_test = np.split(data, [int(.8*len(data))])

df_train, df_valid = np.split(df_trainvalid, [int(.8*len(df_trainvalid))])

X_train = df_train[features]
y_train = df_train['action']

X_valid = df_valid[features]
y_valid = df_valid['action']

X_test = df_test[features]
y_test = df_test['action']


In [5]:
NORMALIZE_NONE = 0
NORMALIZE_MIN_MAX = 1
NORMALIZE_MEAN = 2

def normalize_data(df):
    if NORMALIZE_TYPE == NORMALIZE_MIN_MAX:
        return (df-df.min())/(df.max()-df.min())
    elif NORMALIZE_TYPE == NORMALIZE_MEAN:
        return (df-df.mean())/df.std()
    else:
        return df;
    
NORMALIZE_TYPE = NORMALIZE_MEAN

X_train = normalize_data(X_train)
X_valid = normalize_data(X_valid)
X_test = normalize_data(X_test)

X_trainvalid = normalize_data(df_trainvalid[features])
y_trainvalid = df_trainvalid['action']

In [6]:

from sklearn.decomposition import PCA

pca = PCA(n_components=50).fit(X_train)
X_train = pca.transform(X_train)
X_valid = pca.transform(X_valid)
X_test = pca.transform(X_test)

X_trainvalid = pca.transform(X_trainvalid)

In [7]:
# We create the XGboost-specific DMatrix data format from the numpy array. 
# This data structure is optimised for memory efficiency and training speed
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_valid, label=y_valid)

In [8]:
del data, df_train, df_valid, df_trainvalid
gc.collect()

165

In [9]:
# The objective function is passed an Optuna specific argument of trial
def objective(trial):
    
# params specifies the XGBoost hyperparameters to be tuned
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 200, 600),
        'max_depth': trial.suggest_int('max_depth', 10, 25),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.1),
        'subsample': trial.suggest_uniform('subsample', 0.50, 1),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.50, 1),
        'gamma': trial.suggest_int('gamma', 0, 10),
        'tree_method': 'gpu_hist',  
        'objective': 'binary:logistic'
    }
    
    bst = xgb.train(params, dtrain)
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
# trials will be evaluated based on their accuracy on the test set
    accuracy = accuracy_score(y_valid, pred_labels)
    return accuracy



In [10]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=25, timeout=600)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2021-03-11 12:35:08,368] A new study created in memory with name: no-name-3faf5fe9-173d-4d14-a069-4b4d2c1e369b


[12:35:08] WARNING: ../src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:35:09] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2021-03-11 12:35:11,355] Trial 0 finished with value: 0.5063815739757465 and parameters: {'n_estimators': 211, 'max_depth': 15, 'learning_rate': 0.02452998267512899, 'subsample': 0.504538123835391, 'colsample_bytree': 0.77820350802674, 'gamma': 8}. Best is trial 0 with value: 0.5063815739757465.


[12:35:11] WARNING: ../src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:35:12] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2021-03-11 12:35:26,175] Trial 1 finished with value: 0.504651449526105 and parameters: {'n_estimators': 388, 'max_depth': 18, 'learning_rate': 0.040304793699077225, 'subsample': 0.6395035247129126, 'colsample_bytree': 0.6618539275128126, 'gamma': 1}. Best is trial 0 with value: 0.5063815739757465.


...

[12:38:01] WARNING: ../src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:38:01] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2021-03-11 12:38:03,037] Trial 23 finished with value: 0.5078849924630211 and parameters: {'n_estimators': 455, 'max_depth': 12, 'learning_rate': 0.03348268409324293, 'subsample': 0.7941644209447025, 'colsample_bytree': 0.6766636649761969, 'gamma': 1}. Best is trial 17 with value: 0.5095753439368087.


[12:38:03] WARNING: ../src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:38:03] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2021-03-11 12:38:10,576] Trial 24 finished with value: 0.506210550179575 and parameters: {'n_estimators': 525, 'max_depth': 16, 'learning_rate': 0.05039179316980943, 'subsample': 0.8754518282380466, 'colsample_bytree': 0.7632230179714273, 'gamma': 3}. Best is trial 17 with value: 0.5095753439368087.


Number of finished trials:  25
Best trial:
  Value: 0.5095753439368087
  Params: 
    n_estimators: 550
    max_depth: 10
    learning_rate: 0.06778869537300215
    subsample: 0.9211624171838538
    colsample_bytree: 0.8557756580958944
    gamma: 0


In [11]:
best_params = trial.params
best_params['tree_method'] = 'gpu_hist' 
best_params['objective'] = 'binary:logistic'

# Fit the XGBoost classifier with optimal hyperparameters
optimal_clf = xgb.XGBClassifier(**best_params)
                                
optimal_clf.fit(X_trainvalid, y_trainvalid)

[12:38:12] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8557756580958944, gamma=0,
              gpu_id=0, importance_type='gain', interaction_constraints='',
              learning_rate=0.06778869537300215, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=550, n_jobs=2, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=0.9211624171838538, tree_method='gpu_hist',
              validate_parameters=1, verbosity=None)

In [12]:
prediction = optimal_clf.predict(X_test)
result_df = pd.DataFrame({'Date': df_test['date'], 'Weight': df_test['weight'],
                          'Resp': df_test['resp'], 'Action': prediction})

result_df['P'] = result_df['Weight']*result_df['Resp']*result_df['Action']
result_groupby_days = result_df[['Date', 'P']].groupby('Date').sum().reset_index()

p = result_groupby_days['P'].values

t = (np.sum(p)/(np.sqrt(np.sum(p**2))))*np.sqrt(250/len(p))

u = min(max(t, 0), 6) * np.sum(p)

print(f"Utility score is: {u:.3f}")

Utility score is: 1447.022
